In [23]:
import requests
from scrapy import Selector 
import pandas as pd
import selenium
from lets_plot import *
LetsPlot.setup_html()
from tqdm.notebook import tqdm

### Scrapping URL

In [24]:
url="https://www.azmovies.net/genre/16/animation"
response = requests.get(url)
response

<Response [200]>

In [25]:
sel1 = Selector(text = response.text)
sel1

<Selector query=None data='<html lang="en-US" prefix="og: http:/...'>

### Using Requests package 

In [26]:
import requests
import re

# URL to scrape
url = "https://www.azmovies.net/genre/16/animation"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

In [27]:
# Fetch the webpage
response = requests.get(url, headers=headers)

if response.status_code == 200:
    page_content = response.text   

### Scrapping the links of the pages (First 60 Pages)

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Set up the Selenium WebDriver (you can use Chrome or another browser)
driver = webdriver.Chrome()

# Function to scrape links from each page
def scrape_page_links(url):
    driver.get(url)
    # Wait for the page to load (adjust the waiting time as needed)
    time.sleep(2)

    # Find all the <a> tags inside <h2> tags inside div with class 'gridinfo'
    links = driver.find_elements(By.CSS_SELECTOR, "div.gridinfo > h2 > a")

    # Extract the href attribute from each <a> tag
    hrefs = [link.get_attribute('href') for link in links]
    return hrefs

# Base URL for the page
base_url = 'https://www.azmovies.net/genre/16/animation'  # Replace with your base URL

# List to hold all the links from all pages
all_links = []

# Loop through the first 60 pages
for page in range(1, 61):
    # Construct the URL for each page (assuming pagination is based on a page number in the URL)
    page_url = f'{base_url}/{page}'  # Adjust this URL structure based on how your site handles pagination
    print(f'Scraping page {page_url}')

    # Scrape links from the current page
    page_links = scrape_page_links(page_url)

    # Add the links from this page to the list of all links
    all_links.extend(page_links)

# Print all the links scraped from all pages
print(f'Total links scraped: {len(all_links)}')
for link in all_links:
    print(link)

# Close the browser
driver.quit()

Scraping page https://www.azmovies.net/genre/16/animation/1
Scraping page https://www.azmovies.net/genre/16/animation/2
Scraping page https://www.azmovies.net/genre/16/animation/3
Scraping page https://www.azmovies.net/genre/16/animation/4
Scraping page https://www.azmovies.net/genre/16/animation/5
Scraping page https://www.azmovies.net/genre/16/animation/6
Scraping page https://www.azmovies.net/genre/16/animation/7
Scraping page https://www.azmovies.net/genre/16/animation/8
Scraping page https://www.azmovies.net/genre/16/animation/9
Scraping page https://www.azmovies.net/genre/16/animation/10
Scraping page https://www.azmovies.net/genre/16/animation/11
Scraping page https://www.azmovies.net/genre/16/animation/12
Scraping page https://www.azmovies.net/genre/16/animation/13
Scraping page https://www.azmovies.net/genre/16/animation/14
Scraping page https://www.azmovies.net/genre/16/animation/15
Scraping page https://www.azmovies.net/genre/16/animation/16
Scraping page https://www.azmovie

In [29]:
len(all_links)

1200

In [30]:
all_links[0]

'https://www.azmovies.net/movie/famiglia-imbarazzi-la-maledizione-dello-zoccolo'

In [33]:
# Lists to store scraped data
titles = []
genres = []
directors = []
writers = []
languages = []
production_countries = []
production_companies = []
user_ratings = []
release_dates = []
run_times = []
imdb_ratings = []
metascores = []
rotten_tomatoes_scores = []

for link in all_links:
    try:
        res = requests.get(link, timeout=30)  # Added timeout for reliability
        res.raise_for_status()  # Raise an error for bad responses
        sel = Selector(text=res.text)

        # Extract data with defaults
        tit = sel.css('.product-title::text').get() or "N/A"
        gen = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:nth-child(1) > span > a::text").getall() or []
        dire = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:nth-child(8) > a span::text").getall() or []
        sw = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:nth-child(9) > a span::text").getall() or []
        lang = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:contains(Language)::text").get()
        r_time = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:contains('Runtime')::text").get()
        pcountry = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:nth-child(7)::text").get() or "N/A"
        # pc = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:nth-child(6)::text").getall() 
        pc = sel.css('body > div.headpage > div > div > div.headpageinfo > ul > li:contains("Production Company")::text').get()
        ur = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:nth-child(3)> span > span:nth-child(2)::text").get() or "N/A"
        rat = sel.css('.ext-ratings::text').getall() or []
        rel_date = sel.css("body > div.headpage > div > div > div.headpageinfo > ul > li:nth-child(2)::text").get() or "N/A"

        # Data Cleaning
        lang = lang.strip() if lang else "N/A"  
        r_time = r_time.strip() if r_time else "N/A"  
        # pc = pc.strip() if pc else "N/A"
        pcountry = pcountry.strip()
        rel_date = rel_date.strip()
        rat = [r.strip().replace('%', '') for r in rat if r.strip()]  # Remove whitespace and %

        # Initialize rating variables
        imdb_rating, metascore, rotten_tomatoes = None, None, None

        # Process ratings safely
        for i, value in enumerate(rat):
            try:
                if i == 0:
                    imdb_rating = float(value)
                elif i == 1:
                    metascore = float(value)
                elif i == 2:
                    rotten_tomatoes = int(value)
            except ValueError:
                pass  # Ignore errors and keep as None

        # Append data to lists
        titles.append(tit)
        genres.append(", ".join(gen))  # Convert list to string
        directors.append(", ".join(dire))
        writers.append(", ".join(sw))
        languages.append(lang) 
        production_countries.append(pcountry)
        production_companies.append(pc)
        user_ratings.append(ur)
        release_dates.append(rel_date)
        run_times.append(r_time)  
        imdb_ratings.append(imdb_rating)
        metascores.append(metascore)
        rotten_tomatoes_scores.append(rotten_tomatoes)

    except requests.exceptions.RequestException as e:
        print(f"Failed to scrape {link}: {e}")


### Creating Dictionary

In [34]:
import pandas as pd

# Create a dictionary with the collected data
anime_data = {
    "Title": titles,
    "Genre": genres,
    "Director": directors,
    "Writer": writers,
    "Language": languages,
    "Production Country": production_countries,
    "Production Company": production_companies,
    "User Rating": user_ratings,
    "Release Date": release_dates,
    "Runtime": run_times,
    "IMDB Rating": imdb_ratings,
    "Metascore": metascores,
    "Rotten Tomatoes Score": rotten_tomatoes_scores,
}

# Convert to DataFrame
anime_df = pd.DataFrame(anime_data)

# Display first few rows
print(anime_df.head())

# Optionally, save to a CSV file
anime_df.to_csv("animation_movies_data.csv", index=False)

                                               Title  \
0  Famiglia Imbarazzi - La maledizione dello zocc...   
1                         Thelma's Perfect Birthday    
2                    Batman Ninja vs. Yakuza League    
3                                              Flow    
4                                         Green Man    

                           Genre                        Director  \
0              Animation, Comedy                                   
1              Animation, Family                                   
2              Animation, Action  Jumpei Mizusaki, Shinji Takagi   
3   Drama, Comedy, Horror, Crime                                   
4  Romance, Animation, Adventure                                   

             Writer  Language               Production Country  \
0                    Italiano                                    
1                     English                                    
2  Kazuki Nakashima       日本語  Japan, United States of A

In [35]:
anime_df

,Title,Genre,Director,Writer,Language,Production Country,Production Company,User Rating,Release Date,Runtime,IMDB Rating,Metascore,Rotten Tomatoes Score
0,Famiglia Imbarazzi - La maledizione dello zocc...,"Animation, Comedy",,,Italiano,,Euphorica Srl,N/A,2025-03-20,1h 36min,NaN,NaN,NaN
1,Thelma's Perfect Birthday,"Animation, Family",,,English,,Rija Films,N/A,2025-03-21,1h 11min,NaN,NaN,NaN
2,Batman Ninja vs. Yakuza League,"Animation, Action","Jumpei Mizusaki, Shinji Takagi",Kazuki Nakashima,日本語,"Japan, United States of America",Kamikaze Douga,6,2025-03-17,1h 30min,NaN,NaN,NaN
3,Flow,"Drama, Comedy, Horror, Crime",,,English,,None,8,1996-01-01,1h 20min,4.9,NaN,NaN
4,Green Man,"Romance, Animation, Adventure",,,N/A,N/A,None,N/A,2025-03-23,0h 7min,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,Scooby-Doo! Music of the Vampire,"Family, Animation, Mystery, Adventure, Comedy,...",David Block,,English,United States of America,Warner Bros. Animation,7.7,2012-03-07,1h 19min,NaN,NaN,NaN
1196,The Gruffalo's Child,"Adventure, Animation, Family, Fantasy","Uwe Heidschötter, Johannes Weiland",,English,"Germany, United Kingdom, United States of America",Magic Light Pictures,6.8,2011-12-25,0h 26min,7.1,NaN,NaN
1197,Fullmetal Alchemist the Movie: The Sacred Star...,"Adventure, Fantasy, Animation, Action, Science...",Kazuya Murata,Yuichi Shinbo,日本語,Japan,BONES,6.8,2011-07-02,1h 50min,6.9,54.0,83.0
1198,TEKKEN: Blood Vengeance,"Action, Animation, Science Fiction",Youichi Mouri,Dai Sato,日本語,Japan,Digital Frontier,6.7,2011-07-26,1h 40min,NaN,NaN,NaN


In [36]:
anime_df.shape

(1200, 13)

In [ ]:
anime_df

,Title,Genre,Director,Writer,Language,Production Country,Production Company,User Rating,Release Date,Runtime,IMDB Rating,Metascore,Rotten Tomatoes Score
0,Famiglia Imbarazzi - La maledizione dello zocc...,"Animation, Comedy",,,Italiano,,Euphorica Srl,N/A,2025-03-20,1h 36min,NaN,NaN,NaN
1,Thelma's Perfect Birthday,"Animation, Family",,,English,,Rija Films,N/A,2025-03-21,1h 11min,NaN,NaN,NaN
2,Batman Ninja vs. Yakuza League,"Animation, Action","Jumpei Mizusaki, Shinji Takagi",Kazuki Nakashima,日本語,"Japan, United States of America",Kamikaze Douga,6,2025-03-17,1h 30min,NaN,NaN,NaN
3,Flow,"Drama, Comedy, Horror, Crime",,,English,,None,8,1996-01-01,1h 20min,4.9,NaN,NaN
4,Green Man,"Romance, Animation, Adventure",,,N/A,N/A,None,N/A,2025-03-23,0h 7min,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,Scooby-Doo! Music of the Vampire,"Family, Animation, Mystery, Adventure, Comedy,...",David Block,,English,United States of America,Warner Bros. Animation,7.7,2012-03-07,1h 19min,NaN,NaN,NaN
1194,The Gruffalo's Child,"Adventure, Animation, Family, Fantasy","Uwe Heidschötter, Johannes Weiland",,English,"Germany, United Kingdom, United States of America",Magic Light Pictures,6.8,2011-12-25,0h 26min,7.1,NaN,NaN
1195,Fullmetal Alchemist the Movie: The Sacred Star...,"Adventure, Fantasy, Animation, Action, Science...",Kazuya Murata,Yuichi Shinbo,日本語,Japan,BONES,6.8,2011-07-02,1h 50min,6.9,54.0,83.0
1196,TEKKEN: Blood Vengeance,"Action, Animation, Science Fiction",Youichi Mouri,Dai Sato,日本語,Japan,Digital Frontier,6.7,2011-07-26,1h 40min,NaN,NaN,NaN


### Saving it as CSV

In [37]:
anime_df.to_csv('D:/Harsshu MBA/ME204/DE-Project/animation_movies_data.csv')